# Machine Learning Engineer Nanodegree
## Reinforcement Learning
## Project: Train a Smartcab to Drive

Welcome to the fourth project of the Machine Learning Engineer Nanodegree! In this notebook, template code has already been provided for you to aid in your analysis of the *Smartcab* and your implemented learning algorithm. You will not need to modify the included code beyond what is requested. There will be questions that you must answer which relate to the project and the visualizations provided in the notebook. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide in `agent.py`.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

-----

## Getting Started
In this project, you will work towards constructing an optimized Q-Learning driving agent that will navigate a *Smartcab* through its environment towards a goal. Since the *Smartcab* is expected to drive passengers from one location to another, the driving agent will be evaluated on two very important metrics: **Safety** and **Reliability**. A driving agent that gets the *Smartcab* to its destination while running red lights or narrowly avoiding accidents would be considered **unsafe**. Similarly, a driving agent that frequently fails to reach the destination in time would be considered **unreliable**. Maximizing the driving agent's **safety** and **reliability** would ensure that *Smartcabs* have a permanent place in the transportation industry.

**Safety** and **Reliability** are measured using a letter-grade system as follows:

| Grade 	| Safety 	| Reliability 	|
|:-----:	|:------:	|:-----------:	|
|   A+  	|  Agent commits no traffic violations,<br/>and always chooses the correct action. | Agent reaches the destination in time<br />for 100% of trips. |
|   A   	|  Agent commits few minor traffic violations,<br/>such as failing to move on a green light. | Agent reaches the destination on time<br />for at least 90% of trips. |
|   B   	| Agent commits frequent minor traffic violations,<br/>such as failing to move on a green light. | Agent reaches the destination on time<br />for at least 80% of trips. |
|   C   	|  Agent commits at least one major traffic violation,<br/> such as driving through a red light. | Agent reaches the destination on time<br />for at least 70% of trips. |
|   D   	| Agent causes at least one minor accident,<br/> such as turning left on green with oncoming traffic.       	| Agent reaches the destination on time<br />for at least 60% of trips. |
|   F   	|  Agent causes at least one major accident,<br />such as driving through a red light with cross-traffic.      	| Agent fails to reach the destination on time<br />for at least 60% of trips. |

To assist evaluating these important metrics, you will need to load visualization code that will be used later on in the project. Run the code cell below to import this code which is required for your analysis.

In [ ]:
# Import the visualization code
import visuals as vs

# Pretty display for notebooks
%matplotlib inline

### Understand the World
Before starting to work on implementing your driving agent, it's necessary to first understand the world (environment) which the *Smartcab* and driving agent work in. One of the major components to building a self-learning agent is understanding the characteristics about the agent, which includes how the agent operates. To begin, simply run the `agent.py` agent code exactly how it is -- no need to make any additions whatsoever. Let the resulting simulation run for some time to see the various working components. Note that in the visual simulation (if enabled), the **white vehicle** is the *Smartcab*.

### Question 1
In a few sentences, describe what you observe during the simulation when running the default `agent.py` agent code. Some things you could consider:
- *Does the Smartcab move at all during the simulation?*
- *What kind of rewards is the driving agent receiving?*
- *How does the light changing color affect the rewards?*  

**Hint:** From the `/smartcab/` top-level directory (where this notebook is located), run the command 
```bash
'python smartcab/agent.py'
```

**Answer:**

- The smartcab didn't move at all. 
- Reward pattern 
    - When the traffic signal is red, then it received +ve reward(between 1 and 2.5).
    - When the signal is green, then it recieved -ve reward(between -4 and -6).
- All rewards are based on the seviarity of the 'traffic violation type'.
    - It is observed that traffic violation (_not moving when the signal is green_) event got penalized badly(magnitude) compared to the correct action (_not moving when the signal is red_).


### Understand the Code
In addition to understanding the world, it is also necessary to understand the code itself that governs how the world, simulation, and so on operate. Attempting to create a driving agent would be difficult without having at least explored the *"hidden"* devices that make everything work. In the `/smartcab/` top-level directory, there are two folders: `/logs/` (which will be used later) and `/smartcab/`. Open the `/smartcab/` folder and explore each Python file included, then answer the following question.

### Question 2
- *In the *`agent.py`* Python file, choose three flags that can be set and explain how they change the simulation.*
- *In the *`environment.py`* Python file, what Environment class function is called when an agent performs an action?*
- *In the *`simulator.py`* Python file, what is the difference between the *`'render_text()'`* function and the *`'render()'`* function?*
- *In the *`planner.py`* Python file, will the *`'next_waypoint()`* function consider the North-South or East-West direction first?*

**Answer:**

**<font color="blue">agent.py</font>**

1. update_delay 
    - It shows the time gap between two actions. By-default it considers 2 seconds between any two events. 
    - Input type : numeric
2. display      
    - It is used to disable the GUI of the simulator
    - Input type : boolean
3. log_metrics  
    - It is used to persist the simulation results to log files(.csv) inside the "logs" folder
    - Input type : boolean
    
**<font color="blue">environment.py</font>**

- 'act(self, agent, action)' function gets invoked when an agent performs an action.

**<font color="blue">simulator.py</font>**
- render_text() : It gets used while displaying the 'simulation generated result' on the console.
- render() : This function gets invoked to display the similar simulation generated result on the PyGame GUI. 

**<font color="blue">planner.py</font>**
* 'next_waypoint()' function checks if the destination is cardinally East or West before checking the North/South shift    

-----
## Implement a Basic Driving Agent

The first step to creating an optimized Q-Learning driving agent is getting the agent to actually take valid actions. In this case, a valid action is one of `None`, (do nothing) `'left'` (turn left), `right'` (turn right), or `'forward'` (go forward). For your first implementation, navigate to the `'choose_action()'` agent function and make the driving agent randomly choose one of these actions. Note that you have access to several class variables that will help you write this functionality, such as `'self.learning'` and `'self.valid_actions'`. Once implemented, run the agent file and simulation briefly to confirm that your driving agent is taking a random action each time step.

### Basic Agent Simulation Results
To obtain results from the initial simulation, you will need to adjust following flags:
- `'enforce_deadline'` - Set this to `True` to force the driving agent to capture whether it reaches the destination in time.
- `'update_delay'` - Set this to a small value (such as `0.01`) to reduce the time between steps in each trial.
- `'log_metrics'` - Set this to `True` to log the simluation results as a `.csv` file in `/logs/`.
- `'n_test'` - Set this to `'10'` to perform 10 testing trials.

Optionally, you may disable to the visual simulation (which can make the trials go faster) by setting the `'display'` flag to `False`. Flags that have been set here should be returned to their default setting when debugging. It is important that you understand what each flag does and how it affects the simulation!

Once you have successfully completed the initial simulation (there should have been 20 training trials and 10 testing trials), run the code cell below to visualize the results. Note that log files are overwritten when identical simulations are run, so be careful with what log file is being loaded!
Run the agent.py file after setting the flags from projects/smartcab folder instead of projects/smartcab/smartcab.


In [ ]:
# Load the 'sim_no-learning' log file from the initial simulation results
vs.plot_trials('sim_no-learning.csv')

### Question 3
Using the visualization above that was produced from your initial simulation, provide an analysis and make several observations about the driving agent. Be sure that you are making at least one observation about each panel present in the visualization. Some things you could consider:
- *How frequently is the driving agent making bad decisions? How many of those bad decisions cause accidents?*
- *Given that the agent is driving randomly, does the rate of reliability make sense?*
- *What kind of rewards is the agent receiving for its actions? Do the rewards suggest it has been penalized heavily?*
- *As the number of trials increases, does the outcome of results change significantly?*
- *Would this Smartcab be considered safe and/or reliable for its passengers? Why or why not?*

**Answer:**

<font color="blue">1. How frequently is the driving agent making bad decisions? how many of those bad decisions cause accidents?</font>
    
- We checked 20 trials and found that on avg the bad decision frequency is : 39.71%. 
- Out of 277 bad decisions, 62(26.32%) caused accident. 

In [ ]:
# Decision Frequency Calculation Logic
import pandas as pd
import os
import ast
from IPython.display import display 
from sklearn.preprocessing import MinMaxScaler
 
data = pd.read_csv("logs/sim_no-learning.csv")

time_spent = data['initial_deadline'] - data['final_deadline']
good_decision_count = data['actions'].apply(lambda x: ast.literal_eval(x)[0])
bad_decision_count= time_spent - good_decision_count
bad_decision_accident_count = data['actions'].apply(lambda x: (ast.literal_eval(x)[3]+ast.literal_eval(x)[4]))
print ("Bad decision frequency : {:0.2f}%".format((bad_decision_count/time_spent).mean()*100))
print ("Percentage of Bad decision caused accidents : {:0.2f}%".format((bad_decision_accident_count/bad_decision_count).mean()*100))
print ("Total bad decision : {:d} , total accidents count {:d} ".format(bad_decision_count.sum(), bad_decision_accident_count.sum()))

<font color="blue"> 2. Given that the agent is driving randomly, does the rate of reliability make sense? </font>

Out of the 10 testing trial, the agent could reach destination only 1 time with a realiability rating F(worst). The realiability of the agent is very poor and it doesn't have any significance due to its random behaviour. 

In [ ]:
# Reference : Training Trials 
display(data.tail(10))


<font color="blue">3. What kind of rewards is the agent receiving for its actions? Do the rewards suggest it has been penalized heavily? </font>



- The agent receives a small positive reward when producing a good action, and a varying amount of negative reward dependent on the severity of the bad action(traffic violations, accidents etc) it has committed and a penalty based on time remaining to reach his destination.
- Please find below scaled( 0 to 1) values of the 'net rewards' which the agent received in 20 trial runs. It clearly shows that in some cases the magnitude of the reward is significantly higher or lower compared to the mean value. 

In [ ]:
net_reward_nd = (data['net_reward'] - data['net_reward'].mean())/data['net_reward'].std()
print ('Scaled - net_reward\n-------------------')
scaler = MinMaxScaler()
scaler.fit(data[['net_reward']])
print(scaler.transform(data[['net_reward']]))
scaler.transform(data[['net_reward']])
print ("----------------\nMean reward(scaled) : {:.2f}".format(scaler.transform(data[['net_reward']]).mean()))

<font color="blue">4.  As the number of trials increases, does the outcome of results change significantly? </font>



- According to the above plots (ie. 'Reliability Frequency', 'Reward per actions', 'Parameter Value' and 'Rate of Reliability' against 'No of Trials'), there is no significant changes in the result with the increasing number of trials. 

<font color="blue">5.  Would this Smartcab be considered safe and/or reliable for its passengers? Why or why not? </font>


- This Smartcab is not be considered as 'safe' due to its random behaviour. 
  In 20 trial runs, this smart cab took 277 bad decisions which caused accidents 62 times.
  
- It is not reliable. 90% of the time, the agent couldn't reach to its destination. 

-----
## Inform the Driving Agent
The second step to creating an optimized Q-learning driving agent is defining a set of states that the agent can occupy in the environment. Depending on the input, sensory data, and additional variables available to the driving agent, a set of states can be defined for the agent so that it can eventually *learn* what action it should take when occupying a state. The condition of `'if state then action'` for each state is called a **policy**, and is ultimately what the driving agent is expected to learn. Without defining states, the driving agent would never understand which action is most optimal -- or even what environmental variables and conditions it cares about!

### Identify States
Inspecting the `'build_state()'` agent function shows that the driving agent is given the following data from the environment:
- `'waypoint'`, which is the direction the *Smartcab* should drive leading to the destination, relative to the *Smartcab*'s heading.
- `'inputs'`, which is the sensor data from the *Smartcab*. It includes 
  - `'light'`, the color of the light.
  - `'left'`, the intended direction of travel for a vehicle to the *Smartcab*'s left. Returns `None` if no vehicle is present.
  - `'right'`, the intended direction of travel for a vehicle to the *Smartcab*'s right. Returns `None` if no vehicle is present.
  - `'oncoming'`, the intended direction of travel for a vehicle across the intersection from the *Smartcab*. Returns `None` if no vehicle is present.
- `'deadline'`, which is the number of actions remaining for the *Smartcab* to reach the destination before running out of time.

### Question 4
*Which features available to the agent are most relevant for learning both **safety** and **efficiency**? Why are these features appropriate for modeling the *Smartcab* in the environment? If you did not choose some features, why are those features* not *appropriate? Please note that whatever features you eventually choose for your agent's state, must be argued for here. That is: your code in agent.py should reflect the features chosen in this answer.
*

NOTE: You are not allowed to engineer new features for the smartcab. 

**Answer:**


- <font color="green">**Safety**</font> - Here is the list of relevant features for learning safety. Without considering these inputs, the agent may take bad desions(traffic violation) and/or cause accidents.

    - waypoint : this feature gives guidence on the driving direction. Reference - [L1 Distance](https://en.wikipedia.org/wiki/Taxicab_geometry)
    - input-light : The Smartcab must learn the significance of various states(in this case red/green) of traffic signals.
    - input-left : This is essential to know while taking the right turn on a red signal. 
    - input-oncoming : The SmartCab should learn this to take proper _'left turn on green signal'_ and _'right turn on red'_.



- <font color="green">**Efficiency**</font>

    - deadline : A driving agent that frequently fails to reach the destination in time would be considered unreliable. It can't aford to take a long route for reaching the destination. To be efficient, it must understand the significance of remaning steps/deadline and select an optimal path accordingly. 
    
    
- <font color="red">**Ignored Values**</font>(in the model)    
    - input['right'] - 
        In this context, (w.r.t the U.S. Right-of-Way rules), the agent(Smartcab) doesn't need to know the intended 'direction of travel' for a vehicle to its right. 
        It is sufficient to know the driving direction of the oncoming and left agents as they operates in the same signal state.
        
        - Reference - [Turning right on red](https://www.youtube.com/watch?v=T3gkt5Xvgd8)
    - deadline - Since deadline is a continuous value, so couldn't be added in the defination of the 'state' variable. It will increase the state space and data/trianing requirement of the model. 

### Define a State Space
When defining a set of states that the agent can occupy, it is necessary to consider the *size* of the state space. That is to say, if you expect the driving agent to learn a **policy** for each state, you would need to have an optimal action for *every* state the agent can occupy. If the number of all possible states is very large, it might be the case that the driving agent never learns what to do in some states, which can lead to uninformed decisions. For example, consider a case where the following features are used to define the state of the *Smartcab*:

`('is_raining', 'is_foggy', 'is_red_light', 'turn_left', 'no_traffic', 'previous_turn_left', 'time_of_day')`.

How frequently would the agent occupy a state like `(False, True, True, True, False, False, '3AM')`? Without a near-infinite amount of time for training, it's doubtful the agent would ever learn the proper action!

### Question 5
*If a state is defined using the features you've selected from **Question 4**, what would be the size of the state space? Given what you know about the environment and how it is simulated, do you think the driving agent could learn a policy for each possible state within a reasonable number of training trials?*  
**Hint:** Consider the *combinations* of features to calculate the total number of states!

**Answer:**

|No|Features|Possible Values|Possible Values Count|
|:--|:--------|:---------------|:-----------------|
|1|waypoint |forward, left, right|3|
|2|inputs['light'] |green, red|2|
|3|inputs['left'] |None, forward, left, right|4|
|4|inputs['oncoming'] |None, forward, left, right|4|

- The size of the total state space is = **96** (3\*2\*4\*4).
- Since the state space size(96) is relatively low, I think the agent can learn an optimal policy for each one of these states within a resonable number of traning trials. 

Reference - [Environment.py](https://github.com/udacity/machine-learning/blob/48178d734ba9e97291498b5ebc57639a71f6548c/projects/smartcab/smartcab/environment.py#L262-L397)


### Update the Driving Agent State
For your second implementation, navigate to the `'build_state()'` agent function. With the justification you've provided in **Question 4**, you will now set the `'state'` variable to a tuple of all the features necessary for Q-Learning. Confirm your driving agent is updating its state by running the agent file and simulation briefly and note whether the state is displaying. If the visual simulation is used, confirm that the updated state corresponds with what is seen in the simulation.

**Note:** Remember to reset simulation flags to their default setting when making this observation!

-----
## Implement a Q-Learning Driving Agent
The third step to creating an optimized Q-Learning agent is to begin implementing the functionality of Q-Learning itself. The concept of Q-Learning is fairly straightforward: For every state the agent visits, create an entry in the Q-table for all state-action pairs available. Then, when the agent encounters a state and performs an action, update the Q-value associated with that state-action pair based on the reward received and the iterative update rule implemented. Of course, additional benefits come from Q-Learning, such that we can have the agent choose the *best* action for each state based on the Q-values of each state-action pair possible. For this project, you will be implementing a *decaying,* $\epsilon$*-greedy* Q-learning algorithm with **no discount factor**. Follow the implementation instructions under each **TODO** in the agent functions.

Note that the agent attribute `self.Q` is a dictionary: This is how the Q-table will be formed. Each state will be a key of the `self.Q` dictionary, and each value will then be another dictionary that holds the *action* and *Q-value*. Here is an example:

```
{ 'state-1': { 
    'action-1' : Qvalue-1,
    'action-2' : Qvalue-2,
     ...
   },
  'state-2': {
    'action-1' : Qvalue-1,
     ...
   },
   ...
}
```

Furthermore, note that you are expected to use a *decaying* $\epsilon$ *(exploration) factor*. Hence, as the number of trials increases, $\epsilon$ should decrease towards 0. This is because the agent is expected to learn from its behavior and begin acting on its learned behavior. Additionally, The agent will be tested on what it has learned after $\epsilon$ has passed a certain threshold (the default threshold is 0.05). For the initial Q-Learning implementation, you will be implementing a linear decaying function for $\epsilon$.

### Q-Learning Simulation Results
To obtain results from the initial Q-Learning implementation, you will need to adjust the following flags and setup:
- `'enforce_deadline'` - Set this to `True` to force the driving agent to capture whether it reaches the destination in time.
- `'update_delay'` - Set this to a small value (such as `0.01`) to reduce the time between steps in each trial.
- `'log_metrics'` - Set this to `True` to log the simluation results as a `.csv` file and the Q-table as a `.txt` file in `/logs/`.
- `'n_test'` - Set this to `'10'` to perform 10 testing trials.
- `'learning'` - Set this to `'True'` to tell the driving agent to use your Q-Learning implementation.

In addition, use the following decay function for $\epsilon$:

$$ \epsilon_{t+1} = \epsilon_{t} - 0.05, \hspace{10px}\textrm{for trial number } t$$

If you have difficulty getting your implementation to work, try setting the `'verbose'` flag to `True` to help debug. Flags that have been set here should be returned to their default setting when debugging. It is important that you understand what each flag does and how it affects the simulation! 

Once you have successfully completed the initial Q-Learning simulation, run the code cell below to visualize the results. Note that log files are overwritten when identical simulations are run, so be careful with what log file is being loaded!

In [ ]:
# Load the 'sim_default-learning' file from the default Q-Learning simulation
vs.plot_trials('sim_default-learning.csv')

### Question 6
Using the visualization above that was produced from your default Q-Learning simulation, provide an analysis and make observations about the driving agent like in **Question 3**. Note that the simulation should have also produced the Q-table in a text file which can help you make observations about the agent's learning. Some additional things you could consider:  
- *Are there any observations that are similar between the basic driving agent and the default Q-Learning agent?*
- *Approximately how many training trials did the driving agent require before testing? Does that number make sense given the epsilon-tolerance?*
- *Is the decaying function you implemented for $\epsilon$ (the exploration factor) accurately represented in the parameters panel?*
- *As the number of training trials increased, did the number of bad actions decrease? Did the average reward increase?*
- *How does the safety and reliability rating compare to the initial driving agent?*

**Answer:**

1. <font color="blue">Observations that are similar between the basic driving agent and the default Q-Learning agent</font>
    - In both the casese, Reliability rating is poor(F). Here, 'Safety rating' is slightly better than that of the basic(random) agent model.

2. <font color="blue">No. of training trials</font>
    - 20 training trials are used beforing starting the testing trials. The epsilon value became less than 0.5 starting from 0.95 following the linear decaying function. When the total training trial count is 20 then the epsilon value(accroding datasheet - 3.191891195797325e-16) became less the the default tolerance(0.05).
    
3. <font color="blue">decaying function</font>
    - Yes, the parameter panel perfectly represented the linera decaying function. 
    
4. <font color="blue">number of bad actions decrease? Did the average reward increas ? w.r.t trial count</font>
    - Yes, The number of bad actions( and its related 'traffic violation' and 'accident' frequencies) decreased along with the increasing number of trials. 
    - The 'avg reward per action' steadily increased from -4 to 0 
    
5. <font color="blue">Safety and reliability rating compare to the initial driving agent</font>
    - 'Safety Rating' has remained same (ie. 'F')
    - 'Reliability Rating' has increased to **'B'** from 'F'.
    'Random action' helped the model to do data exploration and to increase the 'Safety' and 'Reliability' ratings. 


-----
## Improve the Q-Learning Driving Agent
The third step to creating an optimized Q-Learning agent is to perform the optimization! Now that the Q-Learning algorithm is implemented and the driving agent is successfully learning, it's necessary to tune settings and adjust learning paramaters so the driving agent learns both **safety** and **efficiency**. Typically this step will require a lot of trial and error, as some settings will invariably make the learning worse. One thing to keep in mind is the act of learning itself and the time that this takes: In theory, we could allow the agent to learn for an incredibly long amount of time; however, another goal of Q-Learning is to *transition from experimenting with unlearned behavior to acting on learned behavior*. For example, always allowing the agent to perform a random action during training (if $\epsilon = 1$ and never decays) will certainly make it *learn*, but never let it *act*. When improving on your Q-Learning implementation, consider the implications it creates and whether it is logistically sensible to make a particular adjustment.

### Improved Q-Learning Simulation Results
To obtain results from the initial Q-Learning implementation, you will need to adjust the following flags and setup:
- `'enforce_deadline'` - Set this to `True` to force the driving agent to capture whether it reaches the destination in time.
- `'update_delay'` - Set this to a small value (such as `0.01`) to reduce the time between steps in each trial.
- `'log_metrics'` - Set this to `True` to log the simluation results as a `.csv` file and the Q-table as a `.txt` file in `/logs/`.
- `'learning'` - Set this to `'True'` to tell the driving agent to use your Q-Learning implementation.
- `'optimized'` - Set this to `'True'` to tell the driving agent you are performing an optimized version of the Q-Learning implementation.

Additional flags that can be adjusted as part of optimizing the Q-Learning agent:
- `'n_test'` - Set this to some positive number (previously 10) to perform that many testing trials.
- `'alpha'` - Set this to a real number between 0 - 1 to adjust the learning rate of the Q-Learning algorithm.
- `'epsilon'` - Set this to a real number between 0 - 1 to adjust the starting exploration factor of the Q-Learning algorithm.
- `'tolerance'` - set this to some small value larger than 0 (default was 0.05) to set the epsilon threshold for testing.

Furthermore, use a decaying function of your choice for $\epsilon$ (the exploration factor). Note that whichever function you use, it **must decay to **`'tolerance'`** at a reasonable rate**. The Q-Learning agent will not begin testing until this occurs. Some example decaying functions (for $t$, the number of trials):

$$ \epsilon = a^t, \textrm{for } 0 < a < 1 \hspace{50px}\epsilon = \frac{1}{t^2}\hspace{50px}\epsilon = e^{-at}, \textrm{for } 0 < a < 1 \hspace{50px} \epsilon = \cos(at), \textrm{for } 0 < a < 1$$
You may also use a decaying function for $\alpha$ (the learning rate) if you so choose, however this is typically less common. If you do so, be sure that it adheres to the inequality $0 \leq \alpha \leq 1$.

If you have difficulty getting your implementation to work, try setting the `'verbose'` flag to `True` to help debug. Flags that have been set here should be returned to their default setting when debugging. It is important that you understand what each flag does and how it affects the simulation! 

Once you have successfully completed the improved Q-Learning simulation, run the code cell below to visualize the results. Note that log files are overwritten when identical simulations are run, so be careful with what log file is being loaded!

In [ ]:
# Load the 'sim_improved-learning' file from the improved Q-Learning simulation
vs.plot_trials('sim_improved-learning.csv')

### Question 7
Using the visualization above that was produced from your improved Q-Learning simulation, provide a final analysis and make observations about the improved driving agent like in **Question 6**. Questions you should answer:  
- *What decaying function was used for epsilon (the exploration factor)?*
- *Approximately how many training trials were needed for your agent before begining testing?*
- *What epsilon-tolerance and alpha (learning rate) did you use? Why did you use them?*
- *How much improvement was made with this Q-Learner when compared to the default Q-Learner from the previous section?*
- *Would you say that the Q-Learner results show that your driving agent successfully learned an appropriate policy?*
- *Are you satisfied with the safety and reliability ratings of the *Smartcab*?*

**Answer:**

1. Epsilon decay function

    - Function Ref - ϵ=e−at,for 0<a<1
    - **a = 0.002**
    - Code Ref - self.epsilon = math.exp(-0.002 * self.total_trials). Initial value of epsilon is 1. 

2. Training Trials</font>

    - **~1800** training trials are used to train the model. After 1800 trials, the value of epsilon became less than the defined tolerance limit and the number'total bad desision' frequency(and its sub-components like 'major accident') became close to zero. 

3. Learning Rate(alpha) and Epsilon Tolerance 

    - In this experiment, Epsilon tolerance is **0.03** and 'Learning Rate'(alpha) is **0.5**. 
    -'Learning Rate' was used to find the 'weighted mean'(or expected) value of the reward of any given state/action pair. It is used it to update the Q-value. 
    - Reference code - self.Q[state][action] = (1-self.alpha)*self.Q[state][action] + self.alpha*reward
    - 'Epsilon tolerance' - A decaying 'epsilon' parameter was used to give the model a chance of 'leveraging its learning'(exploitation) and explore other possible options by taking some random actions (exploration). Higher values of epsilon forced the model to take random actions with 'epsilon' probability. The decayed(using exp) epsilon value gradually reduced to the tolerance limit and equipped the model to depend more its leraning(max Q-values in any given state-action pair) instead of taking random actions. 
    
    PN - When 'Nos. of Test trials'(n_test) was set to smaller values( like 20/30), then the model worked really well with 'cos' function. However, it didn't work when the test trial values were high. 
    (I need to find a more robust and systemtic strategy to find the optimal parameters. Looking forward to your suggestion on this.)
    
4. Q-Learner when compared to the default Q-Learner

    - Optimized learning(with decaying epsilon factor) is significantly better than the default learning.
    - 'Safety Rating' has increased to 'A+' from 'D'.
    - 'Reliability Rating' has increased to 'A' from 'B'.

5. Measure of successful learning

    - Here, the result of the 100 test trials indicates that the agent successfully learned the appropriate policies.
    
6. Performance tuning/optimization( safety and reliability ratings )

    - Not satisfied with the sub-optimal reliability rating('A). It can/should be improved to make it perfect.

### Define an Optimal Policy

Sometimes, the answer to the important question *"what am I trying to get my agent to learn?"* only has a theoretical answer and cannot be concretely described. Here, however, you can concretely define what it is the agent is trying to learn, and that is the U.S. right-of-way traffic laws. Since these laws are known information, you can further define, for each state the *Smartcab* is occupying, the optimal action for the driving agent based on these laws. In that case, we call the set of optimal state-action pairs an **optimal policy**. Hence, unlike some theoretical answers, it is clear whether the agent is acting "incorrectly" not only by the reward (penalty) it receives, but also by pure observation. If the agent drives through a red light, we both see it receive a negative reward but also know that it is not the correct behavior. This can be used to your advantage for verifying whether the **policy** your driving agent has learned is the correct one, or if it is a **suboptimal policy**.

### Question 8

1. Please summarize what the optimal policy is for the smartcab in the given environment. What would be the best set of instructions possible given what we know about the environment? 
   _You can explain with words or a table, but you should thoroughly discuss the optimal policy._

2. Next, investigate the `'sim_improved-learning.txt'` text file to see the results of your improved Q-Learning algorithm. _For each state that has been recorded from the simulation, is the **policy** (the action with the highest value) correct for the given state? Are there any states where the policy is different than what would be expected from an optimal policy?_ 

3. Provide a few examples from your recorded Q-table which demonstrate that your smartcab learned the optimal policy. Explain why these entries demonstrate the optimal policy.

4. Try to find at least one entry where the smartcab did _not_ learn the optimal policy.  Discuss why your cab may have not learned the correct policy for the given state.

Be sure to document your `state` dictionary below, it should be easy for the reader to understand what each state represents.

**Answer:** 

<font color="blue">1. Optimal policy is for the smartcab in the given environment</font>

Policy defines what actions to take at any given state which an agent can come across. 
An 'optimal policy' represents the best candidate among the set of available policies for any given state. 

**True "Optimal policy* based on the traffic laws**

_On a green light, the agent can_
  - go forward
  - take right turn 
  - take a left turn if there is no oncoming traffic making a right turn or coming straight through the intersection.

_On a red light, the agent can_
  - should not take any action(stop)
  - can take right turn if no oncoming traffic is approaching from its left through the intersection. 
  - and shouldn't take any left turn. 

    
**Implemented max Q value based "Optimal policy"** 

In this case, the agent(Smartcab) receives different visual inputs like traffic signal, intended direction of the car to its left/right/oncoming direction as well as a guidence( like GPS) from an external source(in this case from the planner object). Trial runs helps the agent to learn a possible set of actions on any given state based on a reward function(Q-Value). Eventually, the agent selects the 'best possible action for any given state'(optimal policy) based on the highest Q-Value(weighted mean/expected reward). In this case, the model ignores the long-term reward and concentrates only on the immediate reward while updating the Q-value of each state-action pair. 

Reference - [Reinforcement Learning](http://www.cis.upenn.edu/~cis519/fall2015/lectures/14_ReinforcementLearning.pdf)

<br>
<font color="blue">2. Observations using the 'sim_improved-learning.txt'.Are there any states where the policy is different than what would be expected from an optimal policy? </font><br>
<font color="blue">4. Try to find at least one entry where the smartcab did not learn the optimal policy. Discuss why your cab may have not learned the correct policy for the given state.</font>


- For the state - **('left', 'green', None, right)**, optimal policy should have been **'taking left action'**. 
- It is possible that the agent didn't encounter these states multiple times. As a result, the Q value didn't converge to the expected(maximized) Q-Value. We could have proved it by keeping track of visit-counts for each state-action pair.


|State No|waypoint|light|left|oncoming|forward|right|None|left|Max-Qvalue|Selected Action|Comment|
|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|:--|
|43|left| green| None| right|0.31|0.72|-5.02|-20.69|0.72|right|SubOptimal. Ideally it should have moved forward instead of going further away from the intedended direction
|17|forward|green|right|right|0|0.4|-2.49|-9.57|0.4|right|SubOptimal. The agent should have taken right. 

<font color="blue">3. Provide a few examples from your recorded Q-table which demonstrate that your smartcab learned the optimal policy. Explain why these entries demonstrate the optimal policy. </font>


In [ ]:
data = pd.read_csv("logs/sim_improved-learning-qtable.csv")
display(data.head(n=3))

-----
### Optional: Future Rewards - Discount Factor, `'gamma'`
Curiously, as part of the Q-Learning algorithm, you were asked to **not** use the discount factor, `'gamma'` in the implementation. Including future rewards in the algorithm is used to aid in propagating positive rewards backwards from a future state to the current state. Essentially, if the driving agent is given the option to make several actions to arrive at different states, including future rewards will bias the agent towards states that could provide even more rewards. An example of this would be the driving agent moving towards a goal: With all actions and rewards equal, moving towards the goal would theoretically yield better rewards if there is an additional reward for reaching the goal. However, even though in this project, the driving agent is trying to reach a destination in the allotted time, including future rewards will not benefit the agent. In fact, if the agent were given many trials to learn, it could negatively affect Q-values!

### Optional Question 9
*There are two characteristics about the project that invalidate the use of future rewards in the Q-Learning algorithm. One characteristic has to do with the *Smartcab* itself, and the other has to do with the environment. Can you figure out what they are and why future rewards won't work for this project?*

**Answer:**

The whole idea of Gamma is to signal the agent which actions it takes NOW will lead to large rewards in the future i.e Gamma is the parameter that makes the utility of some neighboring states more than others (hence determining the action of the agent) since these states are closer to the terminal high reward state.

**Factor related to the env**
 
- The high reward terminal state (i.e the destination) changes in every trial. The stating location of our agent also changes in every trial. Thus even if the agent actually somehow figured out which block on the grid was closer to the destination, that information would be useless in the next trial.
- If the agent had a map of the grid in it's memory, and, it could find out all the multiple paths that it could take from where it was to it's intended destination, and, could potentially choose multiple paths without having prior knowledge of which paths were more efficient, then gamma would make sense. By discounting rewards for the longer paths we could make our agent more efficient.

**Factor related to the agent** 

- To calculate the discounted reward, the model can only use the 'waypoint' as input which is not alone sufficient for the prediction. Future states will be heavily dependent on the signal which can't be predicted/considered while calculating the utility( current + discounted reward).

- The agent is just following directions that it is being given and it gets rewarded for following the directions while following the traffic laws. According to the code, the agent doesn't need to know where it is going, in fact the agent does not even know when it has reached the destination. All that is handled externally. In this case, there is no benefits of rewarding the agent for making efficient choices (and hence gamma is meaningless.)



> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

## Further Reading

https://github.com/openai/gym